# Project Pharma: the impacts of producing Lovastatin

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import bw2io as bi
import bw2calc as bc
import bw2data as bd
import bw2analyzer as ba
import os

## Importing and looking at the data

In [ ]:
DATA_DIR = os.path.abspath('..\data')
infile = DATA_DIR + "\lci_pharma_dynamic.csv"

df = pd.read_csv(infile, skiprows=1,usecols=[0,1,2,3,15,18])
df.columns = [col.lower() for col in df.columns]
df = df.rename(columns={
    "time": "time_s",
    'q (l/hr)': "inflow_l_hr", 
    'c,la (gr/l)': "conc_la_gr_l",  # Lactose
    'c,ad  (gr/l)': "conc_ad_gr_l",  # Adenosine
    'q (l/hr).3': "outflow_l_hr",
    'c,lov  (gr/l).2': "conc_lov_gr_l",  # Lovastatin
})
df

In [ ]:
# Keep only data where time is an integer (second)
df_s = df.loc[df['time_s']%1==0].reset_index(drop=True)
df_s = df_s.groupby('time_s').agg('mean')

df_s

In [ ]:
df_s.describe()

In [ ]:
# df.iloc[:,1:].plot()
df_s.iloc[:,1:].plot()

In [ ]:
# sns.pairplot(df_s.iloc[:,1:])

## First steps in LCA
From ChatGPT, it takes between 0.5 to 5 kWh per kilogram.

In [ ]:
# Set the seed for reproducibility
SEED = 42
SIZE = 1000
ELEC_CONS = 5  # kWh/kg

# # Create a random number generator with the specified seed
# rng = np.random.default_rng(SEED)

# # Define the parameters for the triangular distribution
# lower_bound = 0.5
# mode = 2.5
# upper_bound = 5

# # Generate a triangular distribution sample using the rng instance
# triangular_sample = rng.triangular(lower_bound, mode, upper_bound, SIZE)

# # Print the first 10 values of the sample to verify
# print(triangular_sample[:10])

In [ ]:
df_s["inflow_la_kg_hr"] = df_s["inflow_l_hr"] * df_s["conc_la_gr_l"] / 1e3
df_s["inflow_ad_kg_hr"] = df_s["inflow_l_hr"] * df_s["conc_ad_gr_l"] / 1e3
df_s["outflow_lov_kg_hr"] = df_s["outflow_l_hr"] * df_s["conc_lov_gr_l"] / 1e3
df_s["electricity"] = df_s["outflow_lov_kg_hr"] * ELEC_CONS
df_s

## Setting up Brightway
This process might change based on everyone's installed Brightway projects, so I'll comment it out but leave it as an example.

In [ ]:
SEED_DATABASE = "new_project"  # TODO: replace with any name of database containing virgin copy of ecoinvent 3.10

if 'pharma' not in [project.name for project in list(bd.projects)]:
    bd.projects.set_current(SEED_DATABASE)
    bd.projects.copy_project('pharma', switch=True)
else:
    bd.projects.set_current('pharma')

bd.databases

In [ ]:
imp = bi.ExcelImporter(DATA_DIR+"\pharma_database_ab.xlsx")
imp.apply_strategies()
imp.match_database("ecoinvent-3.10-cutoff", fields=('name','unit','location'))
imp.statistics()


There was an issue when first importing aminopyridine production, because there was a name change betwen 3.91 and 3.10 from aminopyridine to aminopyridine production.

In [ ]:
[u for u in imp.unlinked]

In [ ]:
# From the tutorials\brightway\2 - Data IO & contribution analyses.ipynb file,
migration = {
    "fields": ["name", "reference product", "location", "categories"],
    "data": [
        (
            ("chichibabin amination", "aminopyridine", "ROW", None),
            {
                "name":"aminopyridine production, Chichibabin amination",
                "location": "ROW"},
        ),
    ],
}

In [ ]:
bi.Migration(name="ei3.91-3.10").write(data=migration, description="ei 3.91 to 3.10")

In [ ]:
"ei3.91-3.10" in bi.migrations

In [ ]:
bi.Migration("ei3.91-3.10")

In [ ]:
# Apply the migration
imp.data = bi.strategies.migrate_exchanges(
    db=imp.data,
    migration="ei3.91-3.10"
)

In [ ]:
imp.match_database("ecoinvent-3.10-cutoff", fields=('name', 'reference product', 'unit', 'location'))
imp.match_database("biosphere3", fields=('name', 'unit', 'categories'))
imp.statistics()

In [ ]:
if len(list(imp.unlinked)) == 0:
    imp.write_database()

In [ ]:
bd.databases

In [ ]:
db = bd.Database("Showcase")

In [ ]:
# let's list the datasets in our new database "carbon fiber"
[a["name"] for a in db]

In [ ]:
# Different ways to check for methods
# [method for method in bd.methods if "ipcc 2021" in method[0].lower() and "gwp100" in method[2].lower() and "land use" not in method[1].lower()]
filtered_methods = [
    method for method in bd.methods
    if "recipe" in method[0].lower()
    and "midpoint" in method[0].lower()
    and "h" in method[0].lower()
    and any(keyword in method[1].lower() for keyword in ["water use", "climate change", "energy resources", "material resources"])
] #if ("water use" or "climate change" or "energy resources" or "material resources") in method[1].lower()]  #  and "total" in method[1].lower() and "h" in method[0].lower()
# ('ReCiPe 2016 v1.03, midpoint (H) no LT', 'climate change no LT', 'global warming potential (GWP100) no LT'),
# ('ReCiPe 2016 v1.03, midpoint (H) no LT',
#   'water use no LT',
#   'water consumption potential (WCP) no LT'),

methods = filtered_methods[4:]

In [ ]:
activity = db.search('Lovastatin')[0]

In [ ]:
method = ('IPCC 2021', 'climate change', 'global warming potential (GWP100)')
try: 
    lca = bc.LCA({activity:1}, method)  # FIXME: check *methods
    lca.lci()
    lca.lcia()
    lca.score
except AssertionError as err:
    print(err)
    # There was an issue where the methods were from EI3.91
    # bd.projects.migrate_project_25()


In [ ]:
# Contribution analysis: processes
pd.DataFrame(
    [(x, y, z["name"]) for x, y, z in ba.ContributionAnalysis().annotated_top_processes(lca=lca)],
    columns=["score", "quantity", "name"]
)

In [ ]:
# Contribution analysis: elementary flows
pd.DataFrame(
    [(x, y, z["name"]) for x, y, z in ba.ContributionAnalysis().annotated_top_emissions(lca=lca)],
    columns=["score", "quantity", "name"]
)

In [ ]:
from polyviz import treemap
from polyviz import sankey

SAVE_PATH = os.path.abspath("..") + r"\results"
SAVE_PATH

In [ ]:
treemap(
    activity=activity,
    method=method,
    filepath=SAVE_PATH+"\\treemap.html",
)  # uses the graph traversal approach


In [ ]:
_, df = sankey(
    activity=activity,
    level=3,
    cutoff=0.05,
    method=method,
    filepath=SAVE_PATH+"\\sankey.html",
    labels_swap={
        "carbon fiber": "cf.",
        "production": "prod."
    }
)

## Adding temporal backgroud

In [ ]:
from bw_temporalis import easy_timedelta_distribution, easy_datetime_distribution, TemporalisLCA, Timeline, TemporalDistribution
from bw_temporalis.lcia import characterize_methane, characterize_co2
import bw_graph_tools as graph


In [ ]:
df_s["outflow_lov_kg_hr"].plot()

In [ ]:
# Check that the cumulative sum is a good approximation of the integral
print(250 * (0.030+0.028) / 2)
df_s["outflow_lov_kg_hr"].transform('cumsum').loc[250]

# Take the last value of the cumulative sum as the integral
total_amount_per_cycle = df_s["outflow_lov_kg_hr"].transform('cumsum').to_numpy()[-1]
total_amount_per_cycle

In [ ]:
(df_s["outflow_lov_kg_hr"]/total_amount_per_cycle).plot()

In [ ]:
# To make for a more interesting case, assume that the data in seconds is in minutes; cut it so that it sums up to a full day of production; assume each day is a new cycle
min_per_day = 24*60

a = TemporalDistribution(
    date=np.array(df_s.loc[0:min_per_day,:].index, dtype='timedelta64[m]'),  # `M` is months
    amount = np.array((df_s.loc[0:min_per_day,"outflow_lov_kg_hr"]/total_amount_per_cycle))  # (df_s["outflow_lov_kg_hr"]/total_amount_per_cycle)
)

a.graph()  # resolution='s'

In [ ]:
# Test timeline
b = TemporalDistribution(
    date=np.array([1,2,3,4,5], 'timedelta64[D]'),
    amount=(np.ones(5)/5),
)

print(a * b)

(a * b).graph()
# plt.xlim([0,200])

# plt.ylim([0,0.00080])

In [ ]:
activity = db.get((db.search('Lovastatin')[0]).as_dict()['code'])
activity

In [ ]:
exchanges = list(activity.exchanges())
exchanges

In [ ]:
bd.databases

In [ ]:
lovastatin_exchange = [exc for exc in db.search('Lovastatin')[0].exchanges() if "lovastatin" in exc["name"].lower()][0]

# lovastatin_exchange.as_dict()
lovastatin_exchange['TemporalDistribution'] = a
lovastatin_exchange.save()

lovastatin_exchange.as_dict()

# print([exc for exc in activity.exchanges()][0].as_dict())

Nice! We saved a temporal distribution.

In [ ]:
lca2 = bc.LCA({activity:1}, method)
lca2.lci()
lca2.lcia()

# start temporalis
lca2 = TemporalisLCA(lca2)

# build timeline
tl = lca2.build_timeline()
tl.build_dataframe()

In [ ]:
bd.get_node(id=107)

In [ ]:
tl.df.plot(x='date', y='amount', kind='scatter')

In [ ]:
a.graph(resolution="s")

In [ ]:
tl.build_dataframe()#.pivot(index=['date','activity'], columns=['flow'], values=['amount'])

In [ ]:
fig, ax = plt.subplots()
sns.scatterplot(
    data=tl.build_dataframe(),
    x='date',
    y='amount',
    hue='flow',
    ax=ax
)

# h, l = 

In [ ]:
print("technosphere:")
for exc in activity.technosphere():
    print(exc)
print("biosphere:")
for exc in activity.biosphere():
    print(exc)
print("production:")
for exc in activity.production():
    print(exc)

In [ ]:
TemporalDistribution?

In [ ]:
np.array(df_s.index, dtype='datetime64[s]')

In [ ]:
print(np.array([0,1,2,3,4], 'timedelta64[h]').shape)
print((np.ones(5)/5).shape)